In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)

naics_primes = pd.read_csv('NAICS Prime Transactions.csv', low_memory=False)
naics_subs = pd.read_csv('NAICS Sub Transactions.csv', low_memory=False)
psc_primes = pd.read_csv('PSC Prime Transactions.csv', low_memory=False)
psc_subs = pd.read_csv('PSC Sub Transactions.csv', low_memory=False)

primes = pd.concat([naics_primes, psc_primes]).drop_duplicates()
subs = pd.concat([naics_subs, psc_subs]).drop_duplicates()

### Top agencies from USASpending

In [3]:
import pandasql as ps

query="""
SELECT
    awarding_agency_name,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE awarding_agency_name IS NOT NULL
GROUP BY awarding_agency_name
ORDER BY total_obligation DESC, number_of_transactions DESC
"""

top_agencies = ps.sqldf(query, locals())
top_agencies.to_excel('top_agencies.xlsx', index=False)
top_agencies

,awarding_agency_name,total_obligation,number_of_transactions
0,Department of Justice,2.231935e+09,1441
1,Department of Veterans Affairs,4.516496e+08,728
2,Department of Homeland Security,2.631673e+08,59
3,Department of Health and Human Services,1.508402e+08,76
4,Court Services and Offender Supervision Agency,5.530342e+07,13
5,Department of Defense,4.743460e+07,46
6,Federal Trade Commission,6.977478e+06,5
7,Department of Labor,2.742545e+06,6
8,Small Business Administration,1.818884e+06,4
9,U.S. International Development Finance Corpora...,1.757374e+06,3


### How much did your top agencies spend on your products/services over the past fiscal year?

In [4]:
list_top_agencies = top_agencies['awarding_agency_name'][:3].tolist()
list_top_agencies = ', '.join(f"'{agency}'" for agency in list_top_agencies)

# How much are the top 3 agencies spending on NAICS codes
query = f"""
SELECT
    awarding_agency_name,
    naics_code,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies})
GROUP BY awarding_agency_name, naics_code
ORDER BY awarding_agency_name DESC, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_naics_spending = ps.sqldf(query, locals())
top_agencies_naics_spending.to_excel('top_agencies_naics_spending.xlsx', index=False)
top_agencies_naics_spending

,awarding_agency_name,naics_code,total_obligation,number_of_transactions
0,Department of Veterans Affairs,624221,2.190399e+08,401
1,Department of Veterans Affairs,623220,1.507659e+08,276
2,Department of Veterans Affairs,624120,4.330168e+07,4
3,Department of Veterans Affairs,541611,2.663022e+07,9
4,Department of Veterans Affairs,623990,7.317921e+06,5
5,Department of Veterans Affairs,561990,2.161465e+06,1
6,Department of Veterans Affairs,713940,1.022960e+06,16
7,Department of Veterans Affairs,621330,6.574738e+05,3
8,Department of Veterans Affairs,624410,3.172500e+05,2
9,Department of Veterans Affairs,812990,1.879269e+05,5


In [ ]:
# How much are the top 3 agencies spending on PSC
query = f"""
SELECT
    awarding_agency_name,
    product_or_service_code,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies})
    AND product_or_service_code = 'R401'
GROUP BY awarding_agency_name, product_or_service_code
ORDER BY awarding_agency_name DESC, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_psc_spending = ps.sqldf(query, locals())
top_agencies_psc_spending.to_excel('top_agencies_psc_spending.xlsx', index=False)
top_agencies_psc_spending

### What percentage of their awards went to small businesses

In [ ]:
query=f"""
SELECT
    awarding_agency_name,
    contracting_officers_determination_of_business_size,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies}) 
    AND action_date_fiscal_year = '2024'
GROUP BY awarding_agency_name, contracting_officers_determination_of_business_size 
ORDER BY total_obligation DESC
"""

top_agencies_sb_spending = ps.sqldf(query, locals())
top_agencies_sb_spending

In [ ]:
query = """
SELECT 
    sb.awarding_agency_name,
    sb.contracting_officers_determination_of_business_size,
    sb.total_obligation AS size_obligation,
    ta.total_obligation AS agency_total_obligation,
    ROUND((sb.total_obligation * 100.0 / ta.total_obligation), 2) AS percentage_of_total
FROM top_agencies_sb_spending AS sb
JOIN top_agencies AS ta
ON sb.awarding_agency_name = ta.awarding_agency_name
ORDER BY sb.awarding_agency_name, sb.contracting_officers_determination_of_business_size DESC
"""

top_agencies_sb_percentage = ps.sqldf(query, locals())
top_agencies_sb_percentage.to_excel('top_agencies_small_business_spending.xlsx', index=False)
top_agencies_sb_percentage

### Do your top agencies utilize small business set-aside and do your certifications match?

In [ ]:
query = f"""
SELECT
    awarding_agency_name,
    type_of_set_aside,
    SUM(total_dollars_obligated) AS total_obligation,
    ROUND((SUM(total_dollars_obligated) * 100.0 / 
           (SELECT SUM(total_dollars_obligated) 
            FROM primes 
            WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
            AND awarding_agency_name = primes.awarding_agency_name
           )), 2) AS percentage_of_total
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies})
GROUP BY awarding_agency_name, type_of_set_aside
ORDER BY total_obligation DESC
"""

top_agencies_set_aside = ps.sqldf(query, locals())
top_agencies_set_aside.to_excel('top_agencies_set_aside_spending.xlsx', index=False)
top_agencies_set_aside

### Who are your top competitors that are winning those awards? How does your business company compare? 

In [ ]:
query="""
SELECT
    recipient_name,
    recipient_uei,
    SUM(total_dollars_obligated) AS total_obligation,
    count(*) AS number_of_transactions,
    contracting_officers_determination_of_business_size,
    organizational_type
FROM primes
GROUP BY recipient_uei
ORDER BY total_obligation DESC, number_of_transactions DESC
"""

top_recipients = ps.sqldf(query, locals())
top_recipients.to_excel('top_recipients.xlsx', index=False)
top_recipients

### Are there sub-contracting opportunities available from these competitors? 

In [ ]:
primes_awardees = top_recipients['recipient_uei'].tolist()
primes_awardees = ', '.join(f"'{agency}'" for agency in primes_awardees)

query = f"""
SELECT
    prime_awardee_name,
    prime_awardee_uei,
    subawardee_name,
    subawardee_uei,
    SUM(prime_award_total_outlayed_amount) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM subs
WHERE prime_awardee_uei IN ({primes_awardees})
GROUP BY subawardee_uei
ORDER BY total_obligation DESC, number_of_transactions DESC
"""

sub_opportunities = ps.sqldf(query, locals())
sub_opportunities

### How often does the agency utilize Simplified Acquisition Procedures (SAP)?

In [ ]:
query="""
SELECT
    awarding_agency_name,
    simplified_procedures_for_certain_commercial_items,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions,
    ROUND(SUM(total_dollars_obligated) * 100.0 / SUM(SUM(total_dollars_obligated)) OVER (PARTITION BY awarding_agency_name), 2) AS percentage_of_obligation
FROM primes
WHERE 
    awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
    AND simplified_procedures_for_certain_commercial_items != "None"
GROUP BY awarding_agency_name, simplified_procedures_for_certain_commercial_items
"""

top_agencies_sap = ps.sqldf(query, locals())
top_agencies_sap.to_excel('top_agencies_sap_spending.xlsx', index=False)
top_agencies_sap

### Top agencies' preferred buying method

In [ ]:
query="""
SELECT
    awarding_agency_name,
    award_type,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, award_type
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_award_type = ps.sqldf(query, locals())
top_agencies_award_type.to_excel('top_agencies_award_type.xlsx', index=False)
top_agencies_award_type

In [ ]:
query="""
SELECT
    awarding_agency_name,
    idv_type,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, idv_type
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_idv_type = ps.sqldf(query, locals())
top_agencies_idv_type.to_excel('top_agencies_idv_type.xlsx', index=False)
top_agencies_idv_type

In [ ]:
query="""
SELECT
    awarding_agency_name,
    type_of_idc,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, type_of_idc
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_type_of_idc = ps.sqldf(query, locals())
top_agencies_type_of_idc.to_excel('top_agencies_type_of_idc.xlsx', index=False)
top_agencies_type_of_idc

In [ ]:
query="""
SELECT
    awarding_agency_name,
    type_of_contract_pricing,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, type_of_contract_pricing
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_type_of_contract_pricing = ps.sqldf(query, locals())
top_agencies_type_of_contract_pricing.to_excel('top_agencies_type_of_contract_pricing.xlsx', index=False)
top_agencies_type_of_contract_pricing

In [ ]:
query="""
SELECT
    awarding_agency_name,
    purchase_card_as_payment_method,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, purchase_card_as_payment_method
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_purchase_card_as_payment_method = ps.sqldf(query, locals())
top_agencies_purchase_card_as_payment_method.to_excel('top_agencies_purchase_card_as_payment_method.xlsx', index=False)
top_agencies_purchase_card_as_payment_method

#### Research the Prime Contractors that have contracts requiring them to hire sub-contractors

In [ ]:
query = """
WITH totals AS (
    SELECT
        awarding_agency_name,
        subcontracting_plan,
        COUNT(*) AS number_of_transactions,
        SUM(total_dollars_obligated) AS total_obligation
    FROM primes
    WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
    GROUP BY awarding_agency_name, subcontracting_plan
),
total_sum AS (
    SELECT SUM(total_obligation) AS grand_total FROM totals
)
SELECT
    t.awarding_agency_name,
    t.subcontracting_plan,
    t.number_of_transactions,
    t.total_obligation,
    ROUND((t.total_obligation * 100.0 / ts.grand_total), 2) AS percentage_of_total
FROM totals t
CROSS JOIN total_sum ts
ORDER BY t.awarding_agency_name DESC, t.subcontracting_plan DESC
"""
top_agencies_subcontracting_plan = ps.sqldf(query, locals())
top_agencies_subcontracting_plan.to_excel('top_agencies_subcontracting_plan.xlsx', index=False)
top_agencies_subcontracting_plan

In [ ]:
for i in primes.columns:
    print(i)

In [ ]:
set(primes['transaction_description'])

In [ ]:
keywords = ['Halfway Home', 'Foster', 'Safehome', 'Programming', 'Lived Experience']  # Replace with your desired keywords

# Create a case-insensitive search pattern
pattern = '|'.join(keywords)

# Filter the dataset based on keywords in the specified columns
filtered_df = primes[primes[['transaction_description']].apply(
    lambda col: col.str.contains(pattern, case=False, na=False)
).any(axis=1)]

filtered_df